# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [ ]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)
value = []

for sqrt_value in generator:
    value.append(sqrt_value)
    print(sqrt_value)

print("\n")
print(sum(value))

# 8.382332347441762 when limit = 5
# 13th value = 3.605551275463989


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


32.854555867161245


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [1]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

age_1 = []

for person in people_1():
    age_1.append(person["Age"])
    print(person)

total_1 = sum(age_1)
print("\n")
print("Sum of ages: ", sum(age_1))
print("\n")


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

age_2 = []

for person in people_2():
    age_2.append(person["Age"])
    print(person)

total_2 = sum(age_2)
print("\n")
print("Sum of ages: ", sum(age_2))
print("\n")

print("Total ages: ", sum([total_1, total_2]))

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}


Sum of ages:  140


{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


Sum of ages:  213


Total ages:  353


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [11]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

import dlt

In [20]:
pipeline = dlt.pipeline(pipeline_name="homework_data",
						destination='duckdb',
						dataset_name='people')


# run the pipeline with default settings, and capture the outcome
info = pipeline.run(people_1(),
                    table_name="users",
                    write_disposition="replace",
                    primary_key="id")

info = pipeline.run(people_2(),
                    table_name="users",
                    write_disposition="merge",
                    primary_key="id")

# show the outcome
print(info)


import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n http_download table below:")

people_data = conn.sql("SELECT * FROM users").df()
display(people_data)

sum_ages = conn.sql("SELECT SUM(age) FROM users")
print(sum_ages)

Pipeline homework_data load step completed in 0.39 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/homework_data.duckdb location to store data
Load package 1708226262.2778473 is LOADED and contains no failed jobs
Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ users               │
└─────────────────────┘




 http_download table below:


,id,name,age,city,occupation,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,None,1708226261.6049051,mqg86UP7SXYU/w
1,2,Person_2,27,City_A,None,1708226261.6049051,0/kncyA/l96tSA
2,5,Person_5,35,City_B,Job_5,1708226262.2778473,KQwNbFrk8RPIKQ
3,4,Person_4,34,City_B,Job_4,1708226262.2778473,Bxl5wm2y+Zw2pQ
4,3,Person_3,33,City_B,Job_3,1708226262.2778473,xuMUPOfPUglcXQ
5,6,Person_6,36,City_B,Job_6,1708226262.2778473,QJ1Z+Ta8/k097Q
6,8,Person_8,38,City_B,Job_8,1708226262.2778473,LB3M62X7ZZzs9w
7,7,Person_7,37,City_B,Job_7,1708226262.2778473,ggflPgFGC1xsqQ


┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      266 │
└──────────┘



Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX